For our research, we are wondering if killer sudokus are harder to solve for a computer than normal sudokus. 

Killer sudokus are a certain type of sudoku that combine elements of sudokus and kakuro. The new rule for these sudokus is that blocks are caged together with a certain sum value. The indices should be summed together to get to this value.

Killer sudokus typically start out with an empty grid, so you will immediately begin using their new rules to deduce some elements to start with.

We are not sure if it is deterministic

We are taking the killer sudokus from x and from them create normal sudokus

We will do this by

In [1]:
import numpy as np
#import math
import pycosat
import itertools
import csv
#import matplotlib.pyplot as plt

In [2]:
def sum_permutations(summers_needed, sum_total):
    nums = np.array([1,2,3,4,5,6,7,8,9])
    all_perms = list(itertools.combinations(nums, summers_needed))
    sum_perms = [p for p in all_perms if sum(p) == sum_total]
    return sum_perms

In [3]:
def load_blocks(filename, indices):
    blocks = {}
    with open(filename) as f:
        reader = csv.reader(f, delimiter=',')
        
        count = 0
        for line in reader:
            sum_list = [int(i) for i in line]
            block_sum = sum_list[-1]
            summers = sum_list[0:-1]
            for summer in summers:
                blocks[summer] = (summers, block_sum)        
        
    return blocks

In [4]:
def load_sum_rules(blocks):
    rules = []

    for _, (summers, sum_total) in blocks.items():
        summers_needed = len(summers)
        perms = sum_permutations(len(summers), sum_total)
        
        for i in range(summers_needed):
            node_rule = []                

            for j in range(0, len(perms)):
                currentPerm = perms[j]
                for k in currentPerm:
                    rule = str(summers[i]) + str(k)
                    node_rule.append(rule)
            rules.append(node_rule)

    return rules

In [5]:
def load_implication_rules(blocks, indices):
    implication_rules = []
    
    for i in range(indices):
        (summers, sum_total) = blocks[i]
        perms = sum_permutations(len(summers), sum_total)
        perm_list = []
        for perm in perms:
            for value in perm:
                if(not value in perm_list):
                    perm_list.append(value)

        elements = []
        for value in perm_list:
            element = str(i) + str(value)
            elements.append(element)
        
        for element in elements:
            implied_elements = list(elements)
            implied_elements.remove(element)
            
            rule = ["-"+element]
            for implied_element in implied_elements:
                rule_copy = list(rule)
                rule_copy.append("-" + implied_element)
                implication_rules.append(rule_copy)
        
        rule = []
        for value in perm_list:
            rule.append("-" + str(i) + str(value))
            summers_copy = list(summers)
            summers_copy.remove(i)
            for summer in summers_copy:
                rule.append( "-" + str(summer) + str(value))
            implication_rules.append(rule)
            rule = []
            
    return implication_rules
    # probably still has copies cause repetition of elements is not checked yet, not too important

In [6]:
# def load_implication_rules(blocks, indices):
#     for i in range(indices):
#         print()
#         print(i)
#         (summers, sum_total) = blocks[i]
#         perms = sum_permutations(len(summers), sum_total)
#         print(perms)
        
#         other_summers = list(summers)
#         other_summers.remove(i)
        
#         for perm in perms:
#             perm_list = list(perm)
#             print(perm_list)
#             for value in perm_list: # for each possible value what does this imply?
#                 print(value)
                
#                 # GET ALL PERMS OF VALUE AND THEN JUST
#                 # 111 IMPLIES 112 IMPLIES 1116 IMPLIES
#                 other_values = list(perm_list)
#                 other_values.remove(value)
#                 print(other_values)
                
#                 element = str(i) + str(value)
                
#                 print(other_summers)
#                 if(len(other_summers) > 1):
#                     other_summers_perm = itertools.permutations(other_summers)
#                     print('ok', list(other_summers_perm))
#                     #perms indices 0,1,0 etc
#                 for summer in other_summers: #how many other indices there are
#                     elements = []
#                     for p in other_values: # how many other values there are
#                         implied_element = str(summer) + str(p)
#                         elements.append(implied_element)
#                     print(elements)
                
#                 #possible values / not possible values
                
#                 #element ==> perm_list
           
#     return

In [7]:
sudokumatrix = np.loadtxt("./easy_killer_sudoku.txt", dtype=int, delimiter=',')
indices_amount = sudokumatrix.shape[0] * sudokumatrix.shape[1]

blocks = load_blocks("./easy_sudoku_rules.txt", indices_amount)
sum_rules = load_sum_rules(blocks)
print(sum_rules)
implication_rules = load_implication_rules(blocks, indices_amount)
print(implication_rules)

#for sol in pycosat.itersolve(cnf):
    #print sol

[['01', '09', '02', '08', '03', '07', '04', '06'], ['11', '19', '12', '18', '13', '17', '14', '16'], ['01', '09', '02', '08', '03', '07', '04', '06'], ['11', '19', '12', '18', '13', '17', '14', '16'], ['25', '28', '29', '26', '27', '29'], ['35', '38', '39', '36', '37', '39'], ['45', '48', '49', '46', '47', '49'], ['25', '28', '29', '26', '27', '29'], ['35', '38', '39', '36', '37', '39'], ['45', '48', '49', '46', '47', '49'], ['25', '28', '29', '26', '27', '29'], ['35', '38', '39', '36', '37', '39'], ['45', '48', '49', '46', '47', '49'], ['51', '57', '52', '56', '53', '55'], ['61', '67', '62', '66', '63', '65'], ['51', '57', '52', '56', '53', '55'], ['61', '67', '62', '66', '63', '65'], ['71', '79', '72', '78', '73', '77', '74', '76'], ['161', '169', '162', '168', '163', '167', '164', '166'], ['71', '79', '72', '78', '73', '77', '74', '76'], ['161', '169', '162', '168', '163', '167', '164', '166'], ['81', '87', '82', '86', '83', '85'], ['171', '177', '172', '176', '173', '175'], ['81', 